- The model predicts much better results if input 2D points and/or input bounding boxes are provided



In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import gc 
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0) 
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6]) 
    h, w = mask.shape[-2:] 
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1) 
    ax.imshow(mask_image) 
    del mask 
    gc.collect() 
        
def show_masks_on_image(raw_image, masks): 
    plt.imshow(np.array(raw_image)) 
    ax = plt.gca() 
    ax.set_autoscale_on(False) 
    for mask in masks:
        show_mask(mask, ax=ax, random_color=True) 
        plt.axis("off") 
        plt.show() 
    del mask 
    gc.collect()

In [2]:
from transformers import pipeline 
generator = pipeline("mask-generation", model="facebook/sam-vit-huge", device=0)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

# ---


In [3]:

import torch
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")
input_points = [[[450, 600]]]  # 2D location of a window in the image

inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)

masks = processor.image_processor.post_process_masks(
    outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu()
)
scores = outputs.iou_scores

ModuleNotFoundError: No module named 'torch'